In [29]:
from probabilistic_classifier.dataset import create_probabilistic_classifier_dataset_gaussian, create_test_dataset_gaussian, create_batched_tensors
from probabilistic_classifier.probabilistic_classifier import ProbabilisticClassifier
import torch
import torch.nn as nn
from torch.optim import Adam

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.01
num_of_iterations = 100
batch_size = 2048
num_class = 2
num_features = 2

model = ProbabilisticClassifier(num_features, [100, 100], num_class).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

In [31]:
data, label = create_probabilistic_classifier_dataset_gaussian([0, 0], [[1, 0.8], [0.8, 1]], 1000000)
data, label = create_batched_tensors(data, label, batch_size)

In [32]:
running_loss = []
running_avg_loss = 0
num_joint = 0
num_marginal = 0
model.train()
for iter_idx in range(num_of_iterations): 
    for inner_iter_idx, (batched_data, batched_label) in enumerate(zip(data, label)):
        num_marginal = num_marginal + torch.count_nonzero(batched_label)
        num_joint = num_joint + (len(batched_label) - torch.count_nonzero(batched_label))
        batched_data, batched_label = batched_data.to(device), batched_label.to(device)

        optimizer.zero_grad()

        estimated_probabilities = model(batched_data)
        loss = criterion(estimated_probabilities, batched_label)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())
        running_avg_loss = running_avg_loss + loss.item()

        if inner_iter_idx == 0 or (inner_iter_idx + 1) % 50 == 0:
            print('iter: {}, inner iter: {}, avg. loss: {}'.format(iter_idx + 1, inner_iter_idx + 1, running_avg_loss / len(running_loss)))          

iter: 1, inner iter: 1, avg. loss: 0.6974862813949585
iter: 1, inner iter: 50, avg. loss: 0.5752098047733307
iter: 1, inner iter: 100, avg. loss: 0.5686529797315597
iter: 1, inner iter: 150, avg. loss: 0.5668244043986003
iter: 1, inner iter: 200, avg. loss: 0.5653618383407593
iter: 1, inner iter: 250, avg. loss: 0.564599473953247
iter: 1, inner iter: 300, avg. loss: 0.564214870929718
iter: 1, inner iter: 350, avg. loss: 0.5635328321797507
iter: 1, inner iter: 400, avg. loss: 0.5631408533453941
iter: 1, inner iter: 450, avg. loss: 0.5629120510154301
iter: 2, inner iter: 1, avg. loss: 0.562856641107676
iter: 2, inner iter: 50, avg. loss: 0.5627161966626409
iter: 2, inner iter: 100, avg. loss: 0.5625982158050472
iter: 2, inner iter: 150, avg. loss: 0.5626073336937059
iter: 2, inner iter: 200, avg. loss: 0.562412927302982
iter: 2, inner iter: 250, avg. loss: 0.5623419276754975
iter: 2, inner iter: 300, avg. loss: 0.5623216232419467
iter: 2, inner iter: 350, avg. loss: 0.5621155965029838
it

In [33]:
p_joint = (num_joint / (num_joint + num_marginal)).item()
p_marginal = (num_marginal / (num_joint + num_marginal)).item()
print(p_joint, p_marginal)

0.5 0.5


In [175]:
with torch.no_grad():
    model.eval()
    softmax = nn.Softmax(dim=1)
    mi = 0
    for i in range(100000):
        test_data = create_test_dataset_gaussian([0, 0], [[1, 0.8], [0.8, 1]], 256)
        test_data = torch.from_numpy(test_data).to(torch.float32)
        test_data = test_data.to(device)
        estimated_test_probabilities = model(test_data)
        
        estimated_test_probabilities = softmax(estimated_test_probabilities)
        joint, marginal = estimated_test_probabilities[:, 0], estimated_test_probabilities[:, 1]
        mi += torch.sum(torch.log(torch.div(joint, marginal))) / 256
    mi / 100000